# Geoapify Places 5 Cities

In [32]:
# Dependencies
from census import Census
import hvplot.pandas
import time
import requests
import json
import pandas as pd
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import the API key
from config_key import geoapify_key
from config_key import census_key

In [33]:
# Create an instance of the Census library
# 
c = Census(
    census_key, 
    year=2021
)

# Run Census Search to retrieve data on all zip codes (2021 ACS5 Census)
census_data = c.acs5.get(
    (
        "B01003_001E",
        "B17001_002E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column renaming
census_df = census_df.rename(
    columns = {
        "B01003_001E": "Population",
        "B17001_002E": "Poverty Count",
        "zip code tabulation area": "Zipcode",
        #rural vs urban,
        #latitude and latitude
    }
)

In [34]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [35]:
#batch API call
#url = "https://api.geoapify.com/v1/batch?apiKey=geoapify_key
#headers = {"Content-Type": "application/json"}
#data = '{"api":"/v2/places","params":{},"inputs":[{"id":"optional-input-id1","params":{}}]}'
      
#try:
#    resp = requests.post(url, headers=headers, data=data)
#    print(resp.json())
#except requests.exceptions.HTTPError as e:
#    print (e.response.text)

In [36]:
cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]
rows = []

In [37]:
for place in places:
    filters = f"place:{place}"
    categories = "commercial.supermarket"
    #radius = 16100 # 10 miles

    params = {
        "categories":categories,
        "apiKey":geoapify_key,
        "filter":filters #,
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url,params=params)

    # convert response to json
    supermarket_data = response.json()

    # print the response url, avoid doing for public github repos in order to avoid exposing key
    print(response.url)
 

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208


In [38]:
    # Print the json (pretty printed)
    print(json.dumps(supermarket_data, indent=4, sort_keys=True))

    # Extracting 'features' and 'properties' from JSON
    features = supermarket_data.get("features", [])

    # Extracting results from JSON 
    # results = supermarket_data.get("results", []) # Create a list to store rows 

    for feature in features: 
        properties = feature.get("properties",{})
        # Extract relevant information
        name = properties.get("name") 
        lat = properties.get("lat") 
        lon = properties.get("lon") # Append data to the list
        postcode = properties.get("postcode")
        radius = properties.get("radius")
        city = properties.get("city")
        rows.append({"Name": name, "City": city, "Zipcode": postcode, "Latitude": lat, "Longitude": lon}) 
    

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -122.4151303026334,
                    37.76908831549504
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Rainbow Grocery Coop",
                "address_line2": "1745 Folsom Street, San Francisco, CA 94103, United States of America",
                "categories": [
                    "building",
                    "building.commercial",
                    "commercial",
                    "commercial.supermarket",
                    "wheelchair",
                    "wheelchair.yes"
                ],
                "city": "San Francisco",
                "country": "United States",
                "country_code": "us",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                

In [39]:
# Create a DataFrame 
supermarket_df = pd.DataFrame(rows) # Display the DataFrame 

# should not return city other than Napa
supermarket_df

,Name,City,Zipcode,Latitude,Longitude
0,Rainbow Grocery Coop,San Francisco,94103,37.769137,-122.415075
1,Nijiya Market,San Francisco,94115,37.785255,-122.431032
2,Safeway,San Francisco,94116,37.743386,-122.474118
3,Foods Co,San Francisco,94124,37.729808,-122.398821
4,Safeway,San Francisco,94131,37.743571,-122.438944
5,Canyon Market,San Francisco,94131,37.734064,-122.433824
6,Whole Foods Market,San Francisco,94107,37.781150,-122.399779
7,Mollie Stone's,San Francisco,94114,37.760666,-122.436559
8,Grocery Outlet,San Francisco,94110,37.753080,-122.415953
9,The Market,San Francisco,94103,37.776542,-122.416807


In [40]:
supermarket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       20 non-null     object 
 1   City       20 non-null     object 
 2   Zipcode    20 non-null     object 
 3   Latitude   20 non-null     float64
 4   Longitude  20 non-null     float64
dtypes: float64(2), object(3)
memory usage: 928.0+ bytes


In [41]:
supermarket_df.columns

Index(['Name', 'City', 'Zipcode', 'Latitude', 'Longitude'], dtype='object')

In [42]:
supermarket_totals_df = supermarket_df.groupby(['City', 'Zipcode']).size().reset_index(name='Supermarket Count')
# supermarket_totals_df = pd.DataFrame[{"City":supermarket_df["City"],"Zipcode":supermarket_df["Zipcode"],"Supermarket Count":supermarket_df["Supermarket Count"]}].groupby["Zipcode"]
supermarket_totals_df

,City,Zipcode,Supermarket Count
0,San Francisco,94103,2
1,San Francisco,94107,2
2,San Francisco,94109,1
3,San Francisco,94110,2
4,San Francisco,94111,1
5,San Francisco,94112,1
6,San Francisco,94114,2
7,San Francisco,94115,1
8,San Francisco,94116,1
9,San Francisco,94118,1


In [43]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [44]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33774 entries, 0 to 33773
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Population     33774 non-null  float64
 1   Poverty Count  33774 non-null  float64
 2   Zipcode        33774 non-null  object 
dtypes: float64(2), object(1)
memory usage: 791.7+ KB


In [45]:
census_df[census_df['Zipcode']=='92392']

,Population,Poverty Count,Zipcode
31126,61014.0,8939.0,92392


In [46]:
supermarkets_density_final = supermarkets_density_final.fillna(0)

NameError: name 'supermarkets_density_final' is not defined

In [ ]:
supermarkets_density_final = supermarkets_density_final[["Population","Poverty Count"]].astype(int)
supermarkets_density_final

In [ ]:
# Add a Supermarket Count column (Supermarkets/Population)
#Supermarket_Total = supermarkets_density_final["Supermarket Count"].sum().astype(int)
#Population_Total = supermarkets_density_final["Population"].astype(int).groupby(["Zipcode"])
#Supermarket_Density = Supermarket_Total / Population_Total

# Configure the final DataFrame
#supermarkets_density_final = supermarkets_density_final[
#    [
#        "postcode",
#        "population",
#        "supermarket"
#    ]
#]

# Display DataFrame length and sample data
# print(f"Number of rows in the DataFrame: {len(supermarkets_density_final)}")
#supermarkets_density_final.head()

In [ ]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208&limit=100&apiKey=geoapify_key

cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
        


# Set the parameters to search for supermarkets 

#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
#radius = 16100 # 10 miles
#radius = 1610 # 1 mile
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#filters = f"circle:{longitude},{latitude},{radius}"

filters = f"place:{places[2]}"
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
#bias = f"proximity:{longitude},{latitude}"
#limit = 100


# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
#    "limit": limit,
    "filter":filters #,
#    "bias":bias
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Make an API request using the params dictionary
response = requests.get(base_url,params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)    

In [ ]:
#c.acs5.tables()

In [ ]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208&limit=100&apiKey=YOUR_API_KEY

cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
          
#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]


# Set the parameters to search for supermarkets 
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
#radius = 16100 # 10 miles
#radius = 1610 # 1 mile
i = 0

# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#filters = f"circle:{longitude},{latitude},{radius}"

filters = f"place:{places[2]}"
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
#bias = f"proximity:{longitude},{latitude}"
#limit = 100


# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
#    "limit": limit,
    "filter":filters #,
#    "bias":bias
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Make an API request using the params dictionaty
response = requests.get(base_url,params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)    

In [ ]:
    # convert response to json
    supermarket_data = response.json()

    # Print the json (pretty printed)
    print(json.dumps(supermarket_data, indent=4, sort_keys=True))